#  Recycler Simulation

In [1]:
import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt

import seaborn as sns
sns.set_context('talk')

import copy

from scipy.constants import c, e, m_p

import PyHEADTAIL

from FERMImachines import RR

from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.impedances.wakes import CircularResistiveWall, CircularResonator, WakeField, WakeTable

PyHEADTAIL v1.16.0


In [2]:
intensities =np.array([0.5,1,2,3,4])
tune_freqs = []
n_turns = 2048

for i, intensity in enumerate(intensities):
    print(intensity)
    machine = RR(n_segments=1, machine_configuration='53MHz', 
                optics='smooth', printer=PyHEADTAIL.general.printers.SilentPrinter())

    C = machine.circumference

    epsn_x = epsn_y = 2.5e-6*np.pi # in [m.rad]

    bunch = machine.generate_6D_Gaussian_bunch_matched(
        n_macroparticles=int(5e4), intensity=intensity*1e11, 
        epsn_x=epsn_x, epsn_y=epsn_y, sigma_z=0.57)

    n_sigma_z = 2
    n_slices = 50
    uniform_bin_slicer = UniformBinSlicer(n_slices=n_slices, n_sigma_z=n_sigma_z)

    #wake_table = CircularResonator(R_shunt=1e6, frequency=1e9, Q=1)

    wakefile = r'C:\\Users\bengl\Documents\\PyHEADTAIL-playground\\TestWakeFile.dat'
    wake_table = WakeTable(wakefile, ['time', 'dipole_x', 'dipole_y', 'quadrupole_x', 'quadrupole_y','nonsense'],n_turns_wake=5)   # Follow the order of the columns in the wake file here. If you want to exclude a component, change the name e.g. for the dipole y to 'no_dipole_y'.
    wake_field = WakeField(uniform_bin_slicer, wake_table) #, wake_table_k, wake_table_k)
    
    #wake_table = CircularResistiveWall(pipe_radius=5e-2, resistive_wall_length=C, conductivity=3e4, dt_min=1e-3/c, beta=bunch.beta, n_turns_wake=100)
    #wake_field = WakeField(uniform_bin_slicer, wake_table)
    machine.one_turn_map.append(wake_field)
    
    m_x = []
    m_y = []
    
    for i in range(n_turns):
        machine.track(bunch)
        m_x.append(bunch.mean_x())
        m_y.append(bunch.mean_y())
        if i % 100 == 0:
            print(i)

    m_x = np.array(m_x)
    m_y = np.array(m_y)

    max_index = np.argmax(np.abs(np.fft.rfft(m_x)))
    freqs = np.fft.rfftfreq(n_turns)
    max_freq = freqs[max_index]

    tune_freqs.append(max_freq)

plt.plot(intensities, tune_freqs, 'o')
plt.xlabel('intensity e11')
plt.ylabel('tune frequency $Q_x$ [Hz]')
a,b = np.polyfit(intensities, tune_freqs, 1)
plt.plot(intensities,  intensities*a+b, label='linear fit')
plt.show()

0.5


c:\Users\bengl\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\integrate\_quadpack_py.py:1233: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


ValueError: dipole_x does not meet requirements.

In [ ]:
print(tune_freqs[1]-tune_freqs[0])
print(freqs[1]-freqs[0])


-0.00048828125
0.00048828125
